# T81-558: Applications of Deep Neural Networks
**Module 13: Advanced/Other Topics**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 13 Video Material

Main video lecture:

* Part 13.1: Deploying a Model to AWS [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.2: Flask and Deep Learning Web Services [[Video]]() [[Notebook]](t81_558_class_13_02_cloud.ipynb)
* Part 13.3: AI at the Edge: Using Keras on a Mobile Device [[Video]]() [[Notebook]](t81_558_class_13_03_web.ipynb)
* **Part 13.4: When to Retrain Your Neural Network** [[Video]]() [[Notebook]](t81_558_class_13_04_edge.ipynb)
* Part 13.5: Using a Keras Deep Neural Network with a Web Application [[Video]]() [[Notebook]](t81_558_class_13_05_retrain.ipynb)


# Part 13.4: When to Retrain Your Neural Network

* Dataset Shift
* Covariate Shift

![Covariate Shift](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/cov-shift.png "Covariate Shift")
[(graph source)](http://iwann.ugr.es/2011/pdf/InvitedTalk-FHerrera-IWANN11.pdf)

### Measures of Drift

* Drift Detection Method (DDM)  [10], 
* Early Drift Detection Method (EDDM)  [11], 
* Page-Hinkley Test (PHT) [12], 
* Adaptive Windowing (ADWIN)  [13], 
* Paired Learners [14], 
* EWMA for Concept Drift Detection (ECDD) [15], 
* Degree of Drift (DOF) [16], and 
* Statistical Test of Equal Proportions (STEPD) [17]

Others.

* KOLMOGOROV SMIRNOV TWO SAMPLE
* https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence

### Academic/Other Sources

* [A unifying view on dataset shift in classification (paper)](https://rtg.cis.upenn.edu/cis700-2019/papers/dataset-shift/dataset-shift-terminology.pdf)
* [A unifying view on dataset shift in classification (presentation)](http://reframe-d2k.org/img_auth.php/7/7e/Presentation_DatasetShift.pdf)
* [A Comparative Study on Concept Drift Detectors](https://www.researchgate.net/publication/264081451_A_Comparative_Study_on_Concept_Drift_Detectors)
* [Covariate Shift – Unearthing hidden problems in Real World Data Science](https://www.analyticsvidhya.com/blog/2017/07/covariate-shift-the-hidden-problem-of-real-world-data-science/)


Kaggle data set:

* [Sberbank Russian Housing Market](https://www.kaggle.com/c/sberbank-russian-housing-market/data)

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

PATH = "/Users/jheaton/Downloads/sberbank-russian-housing-market"


train_df = pd.read_csv(os.path.join(PATH,"train.csv"))
test_df = pd.read_csv(os.path.join(PATH,"test.csv"))

In [2]:
def preprocess(df):
    for i in df.columns:
        if df[i].dtype == 'object':
            df[i] = df[i].fillna(df[i].mode().iloc[0])
        elif (df[i].dtype == 'int' or df[i].dtype == 'float'):
            df[i] = df[i].fillna(np.nanmedian(df[i]))

    enc = LabelEncoder()
    for i in df.columns:
        if (df[i].dtype == 'object'):
            df[i] = enc.fit_transform(df[i].astype('str'))
            df[i] = df[i].astype('object')

In [3]:
preprocess(train_df)
preprocess(test_df)

In [4]:
train_df.drop('price_doc',axis=1,inplace=True)

### KS-Statistic



In [5]:
from scipy import stats

stats.ks_2samp(train_df['kitch_sq'], train_df['kitch_sq'])

Ks_2sampResult(statistic=-0.0, pvalue=1.0)

In [6]:
stats.ks_2samp(train_df['kitch_sq'], test_df['kitch_sq'])

Ks_2sampResult(statistic=0.25829078867676714, pvalue=0.0)

In [7]:
for col in train_df.columns:
    ks = stats.ks_2samp(train_df[col], test_df[col])
    if ks.pvalue < 0.05 and ks.statistic>0.1:
        print(f'{col}: {ks}')

id: Ks_2sampResult(statistic=1.0, pvalue=0.0)
timestamp: Ks_2sampResult(statistic=0.8982081426022823, pvalue=0.0)
life_sq: Ks_2sampResult(statistic=0.2255084471628891, pvalue=7.29401465948424e-271)
max_floor: Ks_2sampResult(statistic=0.17313454154786817, pvalue=7.82000315371674e-160)
build_year: Ks_2sampResult(statistic=0.3176883950430345, pvalue=0.0)
num_room: Ks_2sampResult(statistic=0.1226755470309048, pvalue=1.8622542043144584e-80)
kitch_sq: Ks_2sampResult(statistic=0.25829078867676714, pvalue=0.0)
state: Ks_2sampResult(statistic=0.13641341252952505, pvalue=2.1968159319271184e-99)
preschool_quota: Ks_2sampResult(statistic=0.2364160801236304, pvalue=1.1710777340471466e-297)
school_quota: Ks_2sampResult(statistic=0.25657342859882415, pvalue=0.0)
raion_build_count_with_material_info: Ks_2sampResult(statistic=0.19083554469945835, pvalue=4.2830715478540455e-194)
build_count_block: Ks_2sampResult(statistic=0.2085099875571384, pvalue=1.2494725963839073e-231)
build_count_wood: Ks_2sampResu

### Other

In [8]:
SAMPLE_SIZE = min(len(train_df),len(test_df))
SAMPLE_SIZE = min(SAMPLE_SIZE,10000)
print(SAMPLE_SIZE)

7662


In [9]:
training_sample = train_df.sample(SAMPLE_SIZE, random_state=49)
testing_sample = test_df.sample(SAMPLE_SIZE, random_state=48)

# Is the data from the training set?
training_sample['source_training'] = 1
testing_sample['source_training'] = 0

In [10]:
# Build combined training set
combined = testing_sample.append(training_sample)
combined.reset_index(inplace=True, drop=True)

# Now randomize
combined = combined.reindex(np.random.permutation(combined.index))
combined.reset_index(inplace=True, drop=True)

In [11]:
# Get ready to train
y = combined['source_training'].values
combined.drop('source_training',axis=1,inplace=True)
x = combined.values

In [12]:
y

array([0, 1, 0, ..., 1, 1, 1])

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(n_estimators = 60, max_depth = 7,min_samples_leaf = 5)
lst = []

for i in combined.columns:
    score = cross_val_score(model,pd.DataFrame(combined[i]),y,cv=2,scoring='roc_auc')
    if (np.mean(score) > 0.75):
        lst.append(i)
        print(i,np.mean(score))

id 1.0
timestamp 0.9594980561181873
full_sq 0.7674396951710962
life_sq 0.881961584869916
build_year 0.8018447748079403
kitch_sq 0.9071654626720796
cafe_sum_500_min_price_avg 0.8436231587222647
cafe_avg_price_500 0.8449888567219528
